This VAE doesn't work yet - I'm still wrestling with a dimensionality error that I suspect stems from the computation of reconstructed loss. More to come!

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

c:\Users\mlisi\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


First we will define our encoder layers:

In [3]:
# Encoder
latent_dim = 32 

encoder_inputs = layers.Input(shape=(18, 11, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_inputs)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(latent_dim * 2)(x)  # Two outputs: mean and log variance of latent distribution

Next we sample from the latent distribution and initialize the model! (This is the 'reparameterization trick'.)

In [4]:
z_mean = x[:, :latent_dim]
z_log_var = x[:, latent_dim:]
epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
z = z_mean + tf.exp(0.5 * z_log_var) * epsilon

encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')

Now we'll do the same for the decoder:

In [49]:
# Decoder
latent_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(128, activation='relu')(latent_inputs)
x = layers.Dense(198, activation='relu')(x)
x = layers.Reshape((9, 22, 1))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# Define the decoder model
decoder = Model(latent_inputs, decoder_outputs, name='decoder')


With the encoder and decoder complete, we can construct the VAE. We can define our VAE as a subclass of the Keras `Model` class.

In [50]:
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

vae = VAE(encoder, decoder)

For a loss function we can sum reconstruction loss (how well the decoded output matches the input) with KL-divergence loss (encourages the latent distribution to be close to a Gaussian).

In [51]:
def vae_loss(inputs, reconstructed):
    reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(inputs - reconstructed), axis=[1, 2, 3]))
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    total_loss = reconstruction_loss + kl_loss
    return total_loss


Now once we load in our data, we're ready to train. We'll preprocess the data in the same way as we did for the CNN (see `grade-classification.ipynb`).

In [52]:
f = np.load("moonboard.npz")
X_train, y_train = f['x_train'], f['y_train']
X_test,  y_test  = f['x_test'],  f['y_test']

X_train = np.delete(X_train, 4099, axis=0)
y_train = np.delete(y_train, 4099)

y_test = y_test - 4 # normalizing to 0-12 for sparse categorical cross-entropy
y_train = y_train - 4

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train.reshape((-1, 18, 11, 1))
X_test = X_test.reshape((-1, 18, 11, 1))

In [53]:
X_train.shape
y_train.shape

(9523,)

In [54]:
vae.compile(optimizer='adam', loss=vae_loss)
batch_size = 32
epochs = 100

vae.fit(X_train, X_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, X_test))

Epoch 1/100


ValueError: in user code:

    File "c:\Users\mlisi\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\mlisi\AppData\Local\Temp\ipykernel_23168\1220263707.py", line 2, in vae_loss  *
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(inputs - reconstructed), axis=[1, 2, 3]))

    ValueError: Dimensions must be equal, but are 18 and 36 for '{{node vae_loss/sub}} = Sub[T=DT_FLOAT](IteratorGetNext:1, vae_4/decoder/conv2d_transpose_14/Sigmoid)' with input shapes: [?,18,11,1], [?,36,88,1].
